<a href="https://colab.research.google.com/github/davidandw190/pytorch-deep-learning-workspace/blob/main/notebooks/mnist_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from torchvision import datasets
from torchvision.transforms import ToTensor


In [ ]:

# Load the MNIST dataset for training
train_data = datasets.MNIST(
    root = 'data',          # Directory where the data will be stored
    train = True,           # Directory where the data will be stored
    transform = ToTensor(), # Transforms the images to PyTorch tensors
    download = True         # Downloads the dataset if not present
)

# Load the MNIST dataset for testing
test_data = datasets.MNIST(
    root = 'data',
    train = False,          # Specifies that this is the test dataset
    transform = ToTensor(),
    download = True
)

In [4]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
test_data


Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

The `train_data.data` attribute contains the raw pixel values of the images in the MNIST training dataset. When using the `datasets.MNIST` class from `torchvision`, this attribute is a `torch.Tensor` of shape (60000, 28, 28), where:
- 60000 is the number of training images.
- 28 is the height of each image in pixels.
- 28 is the width of each image in pixels.


2The `.shape` attribute of a `torch.Tensor` provides the dimensions of the tensor as a tuple. This is similar to the .shape attribute in numpy arrays. Understanding the shape of your data is crucial for ensuring that it is correctly formatted for input into neural networks.

In [6]:
train_data.data.shape


torch.Size([60000, 28, 28])

The `train_data.targets` attribute contains the labels for the images in the MNIST training dataset. Each label corresponds to the digit (0-9) that the image represents. In the case of the MNIST dataset, this attribute is a `torch.Tensor` of shape (60000,), where 60000 is the number of training images.

The `.size()` method of a `torch.Tensor` returns the size of the tensor as a `torch.Size` object, which is a subclass of Python's tuple. This method is useful for quickly inspecting the dimensions of a tensor.

In [ ]:
train_data.targets.size()

torch.Size([60000])

In [7]:
from torch.utils.data import DataLoader

In [9]:
# Create data loaders for training and testing datasets
loaders = {
    'train': DataLoader(train_data,
                        batch_size=100,  # Number of samples per batch to load
                        shuffle=True,    # Shuffle the data at every epoch
                        num_workers=1),  # Number of subprocesses to use

    'test': DataLoader(test_data,
                        batch_size=100,
                        shuffle=True,
                        num_workers=1)
}

In [10]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7a66cae0aaa0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7a66cae0aa10>}

In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

  def __init__(self):
    super(CNN, self).__init__()

    # Convolutional layers
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
    self.conv2_drop = nn.Dropout2d() # Dropout layer for regularization

    # Fully connected layers
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    # Apply first convolutional layer followed by ReLU activation
    # and max pooling
    x = F.relu(F.max_pool2d(self.conv1(x), 2))

    # Apply second convolutional layer, dropout, followed by ReLU
    # activation and max pooling
    x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))

    # Flatten the tensor to fit into fully connected layer
    x = x.view(-1, 320)

    # Apply first fully connected layer followed by ReLU activation
    x = F.relu(self.fc1(x))

    # Apply dropout during training
    x = F.dropout(x, training=self.training)

    # Apply second fully connected layer
    x = self.fc2(x)

    return F.log_softmax(x, dim=1)  # Apply log softmax on the output

In [12]:
import torch

# Set the device to GPU if available, otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model and move it to the specified device
model = CNN().to(device)

# Define the optimizer, here using Adam with a learning rate of 0.001
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function, here using CrossEntropyLoss which is suitable for multi-class classification problems
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()  # Set the model to training mode
    for batch_idx, (data, target) in enumerate(loaders['train']):  # Corrected loader key to 'train'
        data, target = data.to(device), target.to(device)  # Move data and target tensors to the specified device
        optimizer.zero_grad()  # Clear the gradients
        output = model(data)  # Forward pass
        loss = loss_fn(output, target)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

        # Print training status every 25 batches
        if batch_idx % 25 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} '
                  f'({100. * batch_idx / len(loaders["train"]):.0f}%)]\tLoss: {loss.item():.6f}')

def test():
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0

    with torch.no_grad():  # Disable gradient computation for evaluation
        for data, target in loaders['test']:  # Iterate over the test data
            data, target = data.to(device), target.to(device)  # Move data and target tensors to the specified device
            output = model(data)  # Forward pass
            test_loss += loss_fn(output, target).item()  # Accumulate the loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # Count correct predictions

    test_loss /= len(loaders['test'].dataset)  # Calculate average test loss
    accuracy = 100. * correct / len(loaders['test'].dataset)  # Calculate accuracy

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders["test"].dataset)} '
          f'({accuracy:.0f}%)\n')  # Print test results


In [13]:
for epoch in range(1, 11):
  train(epoch)
  test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.318525
Train Epoch: 1 [2500/60000 (4%)]	Loss: 2.143365
Train Epoch: 1 [5000/60000 (8%)]	Loss: 1.477910
Train Epoch: 1 [7500/60000 (12%)]	Loss: 1.098507
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.924714
Train Epoch: 1 [12500/60000 (21%)]	Loss: 0.746772
Train Epoch: 1 [15000/60000 (25%)]	Loss: 0.671529
Train Epoch: 1 [17500/60000 (29%)]	Loss: 0.653538
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.633377
Train Epoch: 1 [22500/60000 (38%)]	Loss: 0.601702
Train Epoch: 1 [25000/60000 (42%)]	Loss: 0.398121
Train Epoch: 1 [27500/60000 (46%)]	Loss: 0.530805
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.527685
Train Epoch: 1 [32500/60000 (54%)]	Loss: 0.484680
Train Epoch: 1 [35000/60000 (58%)]	Loss: 0.304631
Train Epoch: 1 [37500/60000 (62%)]	Loss: 0.377491
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.468544
Train Epoch: 1 [42500/60000 (71%)]	Loss: 0.451001
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.426036
Train Epoch: 1 [47500/60000 (79%)]	Loss: 0.306385
Train Epoc